In [1]:
import csv
import lzstring
from collections import namedtuple, Counter
import json
from memoize import memoize
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import sys
import json

In [2]:
csv.field_size_limit(sys.maxsize)

# [num unique urls, num unique urls typed, total visits, total typed, first visit time, last visit time]
domaininfo = namedtuple('domaininfo', ['num_unique_urls', 'num_unique_urls_typed', 'total_visits', 'total_typed', 'first_visit_time', 'last_visit_time'])

decompressFromEncodedURIComponent = lzstring.LZString().decompressFromEncodedURIComponent

#filepath = 'difficultyselectionexp_may31_11am.csv'
filepath = 'data/difficultyselectionexp_june25_9pm.csv'
reader = csv.DictReader(open(filepath))

def extract_domain_visit_info(domain_visit_info_compressed):
  domain_visit_info = json.loads(decompressFromEncodedURIComponent(domain_visit_info_compressed))
  output = {}
  for k,v in domain_visit_info.items():
    linedata = domaininfo(*v)
    output[k] = linedata
  return output

alldata = []

for alldata_item in reader:
  if alldata_item['selected_difficulty'] not in ['nothing', 'easy', 'medium', 'hard']:
    continue
  if alldata_item['domain_visit_info_compressed'] == None or len(alldata_item['domain_visit_info_compressed']) == 0:
    continue
  alldata_item['domain_visit_info'] = extract_domain_visit_info(alldata_item['domain_visit_info_compressed'])
  alldata.append(alldata_item)


In [3]:
#np.random.shuffle(alldata)
training_data = alldata[:round(len(alldata)*0.8)]
test_data = alldata[round(len(alldata)*0.8):]
print(len(training_data))
print(len(test_data))

346
86


In [4]:
def extract_labels_alldata(data):
  return np.array([line['selected_difficulty'] for line in data])

@memoize
def get_most_common_label():
  label_to_count = Counter()
  for line in training_data:
    label = line['selected_difficulty']
    label_to_count[label] += 1
  sorted_by_count = sorted(label_to_count.items(), key=lambda x: x[1], reverse=True)
  return sorted_by_count[0][0]

@memoize
def get_most_visited_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += info.total_visits
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

cnt = 0
@memoize
def get_most_common_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += 1
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits[:100]]

def get_all_domains():
  domain_to_num_visits = Counter()
  for line in training_data:
    domain_visit_info = line['domain_visit_info']
    for domain,info in domain_visit_info.items():
      domain_to_num_visits[domain] += 1
  sorted_by_num_visits = sorted(domain_to_num_visits.items(), key=lambda x: x[1], reverse=True)
  return [x[0] for x in sorted_by_num_visits]



def get_num_visits_for_domain(domain_visit_info, domain):
  info = domain_visit_info.get(domain, None)
  if info != None:
    return info.total_visits
  return 0

def get_productivity():
    with open ('domain_to_productivity.json') as json_file:
        data = json.load(json_file)
        return data

domain_to_productivity = get_productivity()

def extract_features_for_user(domain_visit_info):
  cnt = 0
  domains = get_all_domains()
  final_features = [0, 0, 0, 0, 0];
  for x in domains:
        if x in domain_to_productivity.keys():
            final_features[domain_to_productivity[x] + 2] += get_num_visits_for_domain(domain_visit_info, x)
    
  #np.array([get_num_visits_for_domain(domain_visit_info, x) for x in domains])
  if np.sum(final_features) >= 1:
    final_features = np.divide(final_features, np.sum(final_features))
  return final_features

def extract_features_alldata(data):
  output = []
  for line in data:
    domain_visit_info = line['domain_visit_info']
    features = extract_features_for_user(domain_visit_info)
    output.append(features)
  return np.array(output)


In [5]:
def get_percent_correct(predicted_labels, actual_labels):
  if len(predicted_labels) != len(actual_labels):
    raise 'need predicted and actual labels to have same lengths'
  total = len(actual_labels)
  correct = 0
  for p,a in zip(predicted_labels, actual_labels):
    if p == a:
      correct += 1
  return correct / total

def test_baseline_classifier():
  most_common_label = get_most_common_label()
  predictions = [most_common_label for line in test_data]
  actual = extract_labels_alldata(test_data)
  percent_correct = get_percent_correct(predictions, actual)
  print('baseline classifier accuracy:', percent_correct)

def test_classifier(clf,features_test, actual, str=None):
  #actual = extract_labels_alldata(test_data)
  #features_test = extract_features_alldata(test_data)
  predictions = clf.predict(features_test)
  percent_correct = get_percent_correct(predictions, actual)
  return percent_correct

def training_error_classifier(clf, str=None):
  actual = extract_labels_alldata(training_data)
  features_train = extract_features_alldata(training_data)
  predictions = clf.predict(features_train)
  percent_correct = get_percent_correct(predictions, actual)
  print(str + ' classifier training accuracy:', round(percent_correct, 2))

def to_int_categorical(dt):
  # {'easy', 'hard', 'medium', 'nothing'}
  cat_dt = []
  for item in dt:
    if item == 'nothing':
      cat_dt.append(0)
    elif item == 'easy':
      cat_dt.append(1)
    elif item == 'medium':
      cat_dt.append(2)
    else:
      cat_dt.append(3)
  return np.array(cat_dt)
    
test_baseline_classifier()

baseline classifier accuracy: 0.4186046511627907


In [6]:
labels_train = extract_labels_alldata(training_data)
features_train = extract_features_alldata(training_data)
labels_test = extract_labels_alldata(test_data)
features_test = extract_features_alldata(test_data)

In [7]:
def classifiers():
    clf = RandomForestClassifier(max_depth=10, random_state=0)
    clf.fit(features_train, labels_train)
    RF = test_classifier(clf, features_test, labels_test, 'RF')

    clf = KNeighborsClassifier(n_neighbors=3, p=1)
    clf.fit(features_train, labels_train)
    KNN = test_classifier(clf, features_test, labels_test, 'KNN')
    return [RF, KNN]

In [8]:
print (classifiers())

[0.3488372093023256, 0.43023255813953487]


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import regularizers

labels_test = to_categorical(to_int_categorical(extract_labels_alldata(test_data)), num_classes=4)
features_test = extract_features_alldata(test_data)

labels_train = to_categorical(to_int_categorical(extract_labels_alldata(training_data)), num_classes=4)
features_train = extract_features_alldata(training_data)

Using TensorFlow backend.


In [10]:
def nn():
    model = Sequential()

    model.add(Dense(300, activation='relu', input_dim=5, kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.001))
    model.add(Dense(300, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.001))
    model.add(Dense(300, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.001))
    model.add(Dense(4, activation='softmax', kernel_regularizer=regularizers.l2(0.001)))

    sgd = SGD(lr=0.05, decay=1e-3, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

    model.fit(features_train, labels_train,
          epochs=500,
          batch_size=32)
    score = model.evaluate(features_test, labels_test, batch_size=32)

    predictions = model.predict(features_test, batch_size=32)
    
    print(predictions)
    print(labels_test)
    print(score)
    return score

In [11]:
print (nn())

Epoch 1/500
346/346 [==============================] - 0s 800us/step - loss: 1.9650 - acc: 0.3728
Epoch 2/500
346/346 [==============================] - 0s 87us/step - loss: 1.9084 - acc: 0.4104
Epoch 3/500
346/346 [==============================] - 0s 87us/step - loss: 1.8902 - acc: 0.4104
Epoch 4/500
346/346 [==============================] - 0s 86us/step - loss: 1.8768 - acc: 0.4104
Epoch 5/500
346/346 [==============================] - 0s 87us/step - loss: 1.8662 - acc: 0.4220
Epoch 6/500
346/346 [==============================] - 0s 92us/step - loss: 1.8462 - acc: 0.4364
Epoch 7/500
346/346 [==============================] - 0s 92us/step - loss: 1.8313 - acc: 0.4306
Epoch 8/500
346/346 [==============================] - 0s 95us/step - loss: 1.8209 - acc: 0.4306
Epoch 9/500
346/346 [==============================] - 0s 96us/step - loss: 1.8087 - acc: 0.4335
Epoch 10/500
346/346 [==============================] - 0s 98us/step - loss: 1.7935 - acc: 0.4335
Epoch 11/500
346/346 [======

346/346 [==============================] - 0s 89us/step - loss: 1.4363 - acc: 0.4393
Epoch 85/500
346/346 [==============================] - 0s 90us/step - loss: 1.4369 - acc: 0.4422
Epoch 86/500
346/346 [==============================] - 0s 94us/step - loss: 1.4344 - acc: 0.4393
Epoch 87/500
346/346 [==============================] - 0s 92us/step - loss: 1.4344 - acc: 0.4451
Epoch 88/500
346/346 [==============================] - 0s 88us/step - loss: 1.4330 - acc: 0.4451
Epoch 89/500
346/346 [==============================] - 0s 88us/step - loss: 1.4291 - acc: 0.4422
Epoch 90/500
346/346 [==============================] - 0s 87us/step - loss: 1.4246 - acc: 0.4509
Epoch 91/500
346/346 [==============================] - 0s 90us/step - loss: 1.4239 - acc: 0.4422
Epoch 92/500
346/346 [==============================] - 0s 87us/step - loss: 1.4218 - acc: 0.4393
Epoch 93/500
346/346 [==============================] - 0s 88us/step - loss: 1.4198 - acc: 0.4393
Epoch 94/500
346/346 [===========

Epoch 167/500
346/346 [==============================] - 0s 83us/step - loss: 1.3427 - acc: 0.4509
Epoch 168/500
346/346 [==============================] - 0s 84us/step - loss: 1.3451 - acc: 0.4509
Epoch 169/500
346/346 [==============================] - 0s 84us/step - loss: 1.3493 - acc: 0.4277
Epoch 170/500
346/346 [==============================] - 0s 84us/step - loss: 1.3460 - acc: 0.4393
Epoch 171/500
346/346 [==============================] - 0s 83us/step - loss: 1.3391 - acc: 0.4566
Epoch 172/500
346/346 [==============================] - 0s 88us/step - loss: 1.3422 - acc: 0.4335
Epoch 173/500
346/346 [==============================] - 0s 84us/step - loss: 1.3383 - acc: 0.4480
Epoch 174/500
346/346 [==============================] - 0s 83us/step - loss: 1.3384 - acc: 0.4364
Epoch 175/500
346/346 [==============================] - 0s 86us/step - loss: 1.3390 - acc: 0.4451
Epoch 176/500
346/346 [==============================] - 0s 88us/step - loss: 1.3353 - acc: 0.4393
Epoch 177/

346/346 [==============================] - 0s 89us/step - loss: 1.3058 - acc: 0.4653
Epoch 250/500
346/346 [==============================] - 0s 84us/step - loss: 1.3068 - acc: 0.4538
Epoch 251/500
346/346 [==============================] - 0s 87us/step - loss: 1.3069 - acc: 0.4566
Epoch 252/500
346/346 [==============================] - 0s 89us/step - loss: 1.3056 - acc: 0.4480
Epoch 253/500
346/346 [==============================] - 0s 86us/step - loss: 1.3073 - acc: 0.4566
Epoch 254/500
346/346 [==============================] - 0s 88us/step - loss: 1.3070 - acc: 0.4509
Epoch 255/500
346/346 [==============================] - 0s 92us/step - loss: 1.3048 - acc: 0.4566
Epoch 256/500
346/346 [==============================] - 0s 85us/step - loss: 1.3045 - acc: 0.4509
Epoch 257/500
346/346 [==============================] - 0s 85us/step - loss: 1.3037 - acc: 0.4566
Epoch 258/500
346/346 [==============================] - 0s 87us/step - loss: 1.3037 - acc: 0.4566
Epoch 259/500
346/346 [=

346/346 [==============================] - 0s 86us/step - loss: 1.2938 - acc: 0.4624
Epoch 332/500
346/346 [==============================] - 0s 85us/step - loss: 1.2893 - acc: 0.4566
Epoch 333/500
346/346 [==============================] - 0s 83us/step - loss: 1.2924 - acc: 0.4480
Epoch 334/500
346/346 [==============================] - 0s 87us/step - loss: 1.2935 - acc: 0.4595
Epoch 335/500
346/346 [==============================] - 0s 85us/step - loss: 1.2889 - acc: 0.4538
Epoch 336/500
346/346 [==============================] - 0s 82us/step - loss: 1.2891 - acc: 0.4393
Epoch 337/500
346/346 [==============================] - 0s 85us/step - loss: 1.2873 - acc: 0.4566
Epoch 338/500
346/346 [==============================] - 0s 89us/step - loss: 1.2887 - acc: 0.4538
Epoch 339/500
346/346 [==============================] - 0s 88us/step - loss: 1.2868 - acc: 0.4653
Epoch 340/500
346/346 [==============================] - 0s 85us/step - loss: 1.2890 - acc: 0.4624
Epoch 341/500
346/346 [=

346/346 [==============================] - 0s 89us/step - loss: 1.2803 - acc: 0.4595
Epoch 414/500
346/346 [==============================] - 0s 84us/step - loss: 1.2824 - acc: 0.4538
Epoch 415/500
346/346 [==============================] - 0s 84us/step - loss: 1.2829 - acc: 0.4480
Epoch 416/500
346/346 [==============================] - 0s 87us/step - loss: 1.2773 - acc: 0.4711
Epoch 417/500
346/346 [==============================] - 0s 86us/step - loss: 1.2790 - acc: 0.4538
Epoch 418/500
346/346 [==============================] - 0s 84us/step - loss: 1.2827 - acc: 0.4653
Epoch 419/500
346/346 [==============================] - 0s 85us/step - loss: 1.2810 - acc: 0.4595
Epoch 420/500
346/346 [==============================] - 0s 86us/step - loss: 1.2745 - acc: 0.4624
Epoch 421/500
346/346 [==============================] - 0s 81us/step - loss: 1.2826 - acc: 0.4566
Epoch 422/500
346/346 [==============================] - 0s 84us/step - loss: 1.2798 - acc: 0.4566
Epoch 423/500
346/346 [=

346/346 [==============================] - 0s 92us/step - loss: 1.2712 - acc: 0.4624
Epoch 496/500
346/346 [==============================] - 0s 91us/step - loss: 1.2719 - acc: 0.4480
Epoch 497/500
346/346 [==============================] - 0s 93us/step - loss: 1.2725 - acc: 0.4653
Epoch 498/500
346/346 [==============================] - 0s 91us/step - loss: 1.2755 - acc: 0.4538
Epoch 499/500
346/346 [==============================] - 0s 87us/step - loss: 1.2729 - acc: 0.4538
Epoch 500/500
86/86 [==============================] - 0s 572us/step
[[0.16302802 0.5778641  0.16884968 0.09025829]
 [0.12084141 0.4429481  0.30870512 0.12750532]
 [0.17855981 0.53597033 0.16888437 0.11658546]
 [0.194393   0.53173804 0.17383176 0.10003731]
 [0.11297444 0.50884545 0.25623602 0.12194414]
 [0.20889467 0.4224812  0.24855123 0.12007283]
 [0.10641503 0.45202097 0.3238846  0.11767939]
 [0.19282855 0.41392654 0.26093757 0.13230738]
 [0.32795227 0.39669144 0.14005926 0.13529696]
 [0.09953754 0.24821247 0.4

In [13]:
stats = {'nothing': {'count': 0, 'avg':0}, 'easy' : {'avg': 0, 'count': 0}, 'medium' : {'avg': 0, 'count': 0},
         'hard' : {'avg': 0, 'count': 0}}
users = [{'nothing': 0, 'easy': 0, 'medium': 0, 'hard': 0, 'count': 0}, 
       {'nothing': 0, 'easy': 0, 'medium': 0, 'hard': 0, 'count': 0}, 
       {'nothing': 0, 'easy': 0, 'medium': 0, 'hard': 0, 'count': 0}, 
       {'nothing': 0, 'easy': 0, 'medium': 0, 'hard': 0, 'count': 0}]

for line in training_data:
    domain_visit_info = line['domain_visit_info']
    cnt = 0
    sm = 0
    domains = get_all_domains()
    final_features = [0, 0, 0, 0, 0];
    for x in domains:
        if x in domain_to_productivity.keys():
            sm += domain_to_productivity[x] * get_num_visits_for_domain(domain_visit_info, x)
            cnt += get_num_visits_for_domain(domain_visit_info, x)
    sm /= cnt
    #print(sm)
    stats[line['selected_difficulty']]['count'] += 1
    stats[line['selected_difficulty']]['avg'] += sm
    if sm >= 1: 
        users[3][line['selected_difficulty']] += 1
        users[3]['count'] += 1
    elif sm >= 0: 
        users[2][line['selected_difficulty']] += 1
        users[2]['count'] += 1
    elif sm >= -1:
        users[1][line['selected_difficulty']] += 1
        users[1]['count'] += 1
    else: 
        users[0][line['selected_difficulty']] += 1
        users[0]['count'] += 1
     
    #hard medium easy nothing 
stats['hard']['avg'] /= stats['hard']['count']
stats['medium']['avg'] /= stats['medium']['count']
stats['easy']['avg'] /= stats['easy']['count']
stats['nothing']['avg'] /= stats['easy']['count']
for i in range(-2, 2):
    users[i + 2]['nothing'] /= users[i + 2]['count'] / 100
    users[i + 2]['easy'] /= users[i + 2]['count'] / 100
    users[i + 2]['medium'] /= users[i + 2]['count'] / 100
    users[i + 2]['hard'] /= users[i + 2]['count'] / 100

print(stats)
print ("based on productivity: ")
print ("peope who rank between -2 and -1 (least productive): ")
print (users[0])

print ("people who rank between -1 and 0:")
print (users[1])

print ("people who rank between 0 and 1:")
print (users[2])
print ("people who rank between 1 and 2:")
print (users[3])
    

{'nothing': {'count': 60, 'avg': -0.1495163101219509}, 'easy': {'avg': -0.3228843698277435, 'count': 142}, 'medium': {'avg': -0.1451729862855142, 'count': 97}, 'hard': {'avg': -0.2366551080729964, 'count': 47}}
based on productivity: 
peope who rank between -2 and -1 (least productive): 
{'nothing': 31.11111111111111, 'easy': 37.77777777777778, 'medium': 20.0, 'hard': 11.11111111111111, 'count': 45}
people who rank between -1 and 0:
{'nothing': 13.043478260869565, 'easy': 45.108695652173914, 'medium': 27.173913043478258, 'hard': 14.67391304347826, 'count': 184}
people who rank between 0 and 1:
{'nothing': 21.21212121212121, 'easy': 37.37373737373738, 'medium': 29.292929292929294, 'hard': 12.121212121212121, 'count': 99}
people who rank between 1 and 2:
{'nothing': 5.555555555555555, 'easy': 27.77777777777778, 'medium': 50.0, 'hard': 16.666666666666668, 'count': 18}
